In [24]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import pathlib
import xml.etree.ElementTree as ET
from matplotlib import cm
import meshio
import trimesh
import igl
from tqdm import tqdm_notebook as tqdm

In [6]:
def resolve_path(path, root: pathlib.Path) -> pathlib.Path:
    path = pathlib.Path(path)
    if path.is_absolute():
        return path
    return (root / path).resolve()


def parse_vtm(path):
    tree = ET.parse(path)
    root = tree.getroot()
    blocks = root.find("vtkMultiBlockDataSet").findall("Block")
    # assert(len(blocks) == 1)
    for block in blocks:
        if block.get("name") == "Volume":
            break
    dataset = block.find("DataSet")
    return resolve_path(dataset.attrib["file"], path.parent)


def parse_pvd(path):
    tree = ET.parse(path)
    root = tree.getroot()
    frames = root.find("Collection").findall("DataSet")
    assert(len(frames) > 0)

    meshes = []
    for f in map(lambda f: f.attrib["file"], frames):
        f = resolve_path(f, path.parent)
        if f.suffix == ".vtm":
            f = parse_vtm(f)
        meshes.append(resolve_path(f, path.parent))

    return meshes

In [22]:
def fix_normals(V, F):
    mesh = trimesh.Trimesh(V, F, process=False, validate=False)
    trimesh.repair.fix_normals(mesh)
    return mesh.faces


def load_mesh(path):
    mesh = meshio.read(path)

    V, I, J, _ = igl.remove_duplicate_vertices(
        mesh.points, np.array([], dtype=int), 1e-7)

    CV = []  # codim vertices
    E = []  # edges
    F = []  # triangles
    for cells in mesh.cells:
        if cells.type == "triangle":
            F.append(J[cells.data])
        elif cells.type == "tetra":
            F.append(fix_normals(V, igl.boundary_facets(J[cells.data])))
        elif cells.type == "line":
            E.append(J[cells.data])
        elif cells.type == "vertex":
            CV.append(J[cells.data])
        else:
            raise Exception("Unsupported cell type: {}".format(cells.type))

    cells = []
    if F:
        cells.append(("triangle", np.vstack(F)))
    if E:
        cells.append(("line", np.vstack(E)))
    if CV:
        cells.append(("vertex", np.vstack(CV)))

    if "solution" in mesh.point_data:
        V += mesh.point_data["solution"][I]

    point_data = dict((k, v[I]) for k, v in mesh.point_data.items())
    # if "E" in point_data:
    #     point_data["is_obstacle"] = (point_data["E"] == 0).flatten()
    # else:
    #     point_data["is_obstacle"] = np.zeros((V.shape[0],), dtype=bool)

    mesh = meshio.Mesh(points=V, cells=cells, point_data=point_data)

    return mesh


In [25]:
sim_files = [
    pathlib.Path("/home/zjf214/scratch/remeshing-project/results/spikes3d/drop-ball/2023_01_22_21_47_29_282/sim.pvd")
]

mesh_sequences = [{
    "name": "misc",
    "meshes": []
}]
for f in sim_files:
    if f.suffix == ".pvd":
        mesh_sequences.append({
            "name": f,
            "meshes": parse_pvd(f)
        })
    else:
        mesh_sequences[0]["meshes"].append(f)

for seq in mesh_sequences:
    if not seq["meshes"]:
        continue

    out_dir = pathlib.Path("../blender") / seq["name"].parents[1].stem
    out_dir.mkdir(parents=True, exist_ok=True)

    for i, mesh_path in enumerate(tqdm(seq["meshes"])):
        mesh = load_mesh(mesh_path)
        meshio.write(out_dir / f"{i}.vtu", mesh)

    meshes = [load_mesh(mesh_path) for mesh_path in seq["meshes"]]

    # TODO: save the meshes

<ipython-input-25-a5ddfb5a8c67>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, mesh_path in enumerate(tqdm(seq["meshes"])):


  0%|          | 0/170 [00:00<?, ?it/s]